In [ ]:
import phd

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Some plotting niceties
phd.plots.setup(figsize=(6, 4))

# Miscellaneous figures

The other notebooks are primarily
for generating figures for the thesis,
grouped by model.
Here, we have code to generate other
miscellaneous figures that aren't direclty tied
to one of the models.

## Chapter 2. Background

### Equal loudness curves

Defined by ISO-226;
values and code from
http://www.mathworks.com/matlabcentral/fileexchange/7028-iso-226-equal-loudness-level-contour-signal.

In [ ]:
def equalloudness(phon):
    """Generates an Equal Loudness Contour as described in ISO 226.

    This function will return the equal loudness contour for
    your desired phon level.  The frequencies evaulated in this
    function only span from 20Hz - 12.5kHz, and only 29 selective
    frequencies are covered.  This is the limitation of the ISO
    standard.

    In addition the valid phon range should be 0 - 90 dB SPL.
    Values outside this range do not have experimental values
    and their contours should be treated as inaccurate.

    If more samples are required you should be able to easily
    interpolate these values using spline().

    Parameters
    ----------
    phon : float
        The phon value in dB SPL that you want the equal
        loudness curve to represent. (1phon = 1dB @ 1kHz)

    Returns
    -------
    freq : ndarray (29,)
        Vector of frequencies that ISO226 evaluates to generate the contour.
    spl : ndarray (29,)
        Sound Pressure Level amplitude returned for
        each of the 29 frequencies evaluated by ISO226.

    Note that they are returned in this order so that they can be passed
    directly to a `plot` call; e.g., `plt.plot(*equalloudness(20))`.

    Notes
    -----
    Original Matlab implementation by Jeff Tackett; see
    http://www.mathworks.com/matlabcentral/fileexchange/7028-iso-226-equal-loudness-level-contour-signal

    Used under BSD license.
    """

    # --- Tables from ISO226
    freq = np.array([20, 25, 31.5, 40, 50, 63, 80, 100, 125, 160, 200, 250,
                     315, 400, 500, 630, 800, 1000, 1250, 1600, 2000, 2500,
                     3150, 4000, 5000, 6300, 8000, 10000, 12500], dtype=float)

    af = np.array([0.532, 0.506, 0.480, 0.455, 0.432, 0.409, 0.387, 0.367,
                   0.349, 0.330, 0.315, 0.301, 0.288, 0.276, 0.267, 0.259,
                   0.253, 0.250, 0.246, 0.244, 0.243, 0.243, 0.243, 0.242,
                   0.242, 0.245, 0.254, 0.271, 0.301])

    Lu = np.array([-31.6, -27.2, -23.0, -19.1, -15.9, -13.0, -10.3, -8.1,
                   -6.2, -4.5, -3.1, -2.0, -1.1, -0.4, 0.0, 0.3, 0.5, 0.0,
                   -2.7, -4.1, -1.0, 1.7, 2.5, 1.2, -2.1, -7.1, -11.2, -10.7,
                   -3.1])

    Tf = np.array([78.5, 68.7, 59.5, 51.1, 44.0, 37.5, 31.5, 26.5, 22.1, 17.9,
                   14.4, 11.4, 8.6, 6.2, 4.4, 3.0, 2.2, 2.4, 3.5, 1.7, -1.3,
                   -4.2, -6.0, -5.4, -1.5, 6.0, 12.6, 13.9, 12.3])

    if phon < 0 or phon > 90:
        raise ValueError("`phon` must be between 0 and 90; got %s" % phon)

    Ln = phon

    # Deriving sound pressure level from loudness level (ISO226, section 4.1)
    Af = (0.00447 * (np.power(10, 0.025*Ln) - 1.15)
          + (0.4 * np.power(np.power(10, ((Tf + Lu) / 10) - 9), af)))
    spl = ((10. / af) * np.log10(Af)) - Lu + 94

    return freq, spl

fig = plt.figure()
for phon in [0] + range(10, 110, 20):
    c = 'r' if phon < 10 or phon > 70 else 'k'
    freq, spl = equalloudness(phon)
    plt.plot(freq, spl, c=c)
    if phon == 0:
        text = "0 phon (just audible)"
    else:
        text = "%d phon" % phon
    plt.text(freq[-1] * 1.2, spl[-1], text, verticalalignment='center')
plt.ylabel("Sound pressure level (dB)")
plt.xlabel("Frequency (Hz)")
plt.xscale('log')
sns.despine()
fig.tight_layout()
phd.plots.savefig(fig, 'background', 'el-curves')

### ERB & Mel scales

In [ ]:
freq_hz = np.linspace(100, 6000, 1000)
erb = phd.filters.erbspace(100, 6000, 1000)
mel = phd.filters.melspace(100, 6000, 1000)

fig, ax1 = plt.subplots()
c1 = next(ax1._get_lines.prop_cycler)['color']
c2 = next(ax1._get_lines.prop_cycler)['color']
ax1.plot(freq_hz, erb, c=c1, label="ERB scale")
ax1.set_ylabel("Equivalent rectangular bandwidth (Hz)", color=c1)
for tl in ax1.get_yticklabels():
    tl.set_color(c1)
ax1.set_xlabel("Center frequency (Hz)")
ax1.set_xscale('log')
sns.despine(ax=ax1)

ax2 = ax1.twinx()
ax2.plot(freq_hz, mel, c=c2, label="Mel scale")
ax2.set_ylabel("Mel", color=c2)
for tl in ax2.get_yticklabels():
    tl.set_color(c2)
sns.despine(ax=ax2, right=False)

fig.tight_layout()
phd.plots.savefig(fig, 'background', 'erb-mel')